# E-Ticaret Lojistik Verileri ile Teslimat Tahmini
## Veri Bilimine Giriş ve Makine Öğrenmesi Uygulaması

---

**Veri Seti:** Christmas Retail Sales – Shipping & Delivery Dataset (Kaggle)

**Amaç:** Bu çalışmada, e-ticaret lojistik verileri üzerinde veri biliminin temel adımlarını uygulayarak, teslimat durumunu (geç/zamanında) tahmin eden bir makine öğrenmesi modeli geliştireceğiz.

---

## 1. Kütüphanelerin Yüklenmesi

Veri analizi ve görselleştirme için gerekli Python kütüphanelerini import ediyoruz.

In [ ]:
# Temel veri işleme kütüphaneleri
import pandas as pd
import numpy as np

# Görselleştirme kütüphaneleri
import matplotlib.pyplot as plt
import seaborn as sns

# Makine öğrenmesi kütüphaneleri
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Uyarıları kapatıyoruz (daha temiz çıktı için)
import warnings
warnings.filterwarnings('ignore')

# Grafik ayarları
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Kütüphaneler başarıyla yüklendi!")

---

## 2. Veri Tanıma

### 2.1 Veri Setinin Genel Amacı

Bu veri seti, Black Friday, Cyber Monday ve Noel dönemlerindeki perakende satış verilerini içermektedir. Sipariş, ürün, kampanya, sevkiyat, teslimat, iade ve takvim bilgilerini kapsayan çok tablolu bir yapıya sahiptir.

### 2.2 Tabloların Yüklenmesi

Excel dosyasındaki tüm tabloları (sheet'leri) yükleyerek inceliyoruz.

In [ ]:
# Excel dosyasının yolu
dosya_yolu = "Christmas_Retail_Sales_and_Marketing_Analysis_Dataset.xlsx"

# Excel dosyasındaki tüm sheet isimlerini okuyalım
excel_dosyasi = pd.ExcelFile(dosya_yolu)
tablo_isimleri = excel_dosyasi.sheet_names

print("Veri setindeki tablolar:")
print("-" * 40)
for i, tablo in enumerate(tablo_isimleri, 1):
    print(f"{i}. {tablo}")

In [ ]:
# Her tabloyu ayrı ayrı DataFrame olarak yükleyelim
tablolar = {}
for tablo_ismi in tablo_isimleri:
    tablolar[tablo_ismi] = pd.read_excel(dosya_yolu, sheet_name=tablo_ismi)

# Kolay erişim için değişkenlere atayalım
order_header = tablolar.get('OrderHeader', tablolar.get('Order Header', None))
order_line = tablolar.get('OrderLine', tablolar.get('Order Line', None))
product = tablolar.get('Product', None)
promotion = tablolar.get('Promotion', None)
fulfillment = tablolar.get('Fulfillment', None)
returns = tablolar.get('Returns', None)
calendar = tablolar.get('Calendar', None)

print("Tablolar başarıyla yüklendi!")

### 2.3 Tabloların İş Sürecindeki Rolü

| Tablo | Açıklama |
|-------|----------|
| **OrderHeader** | Sipariş başlık bilgileri (müşteri, tarih, toplam tutar) |
| **OrderLine** | Sipariş satır detayları (ürün, miktar, fiyat) |
| **Product** | Ürün bilgileri (kategori, marka, fiyat) |
| **Promotion** | Kampanya ve indirim bilgileri |
| **Fulfillment** | Sevkiyat ve teslimat bilgileri (kargo, teslimat durumu) |
| **Returns** | İade bilgileri |
| **Calendar** | Tarih boyutu (tatiller, haftasonları) |

In [ ]:
# Her tablonun satır ve sütun sayılarını gösterelim
print("Tabloların Boyutları")
print("=" * 50)
print(f"{'Tablo':<20} {'Satır':<10} {'Sütun':<10}")
print("-" * 50)

for isim, tablo in tablolar.items():
    satir, sutun = tablo.shape
    print(f"{isim:<20} {satir:<10} {sutun:<10}")

### 2.4 Tabloların İlk Bakışı

Her tablonun ilk birkaç satırına ve yapısına göz atalım.

In [ ]:
# Fulfillment tablosu - Ana odak noktamız (teslimat bilgileri)
print("FULFILLMENT TABLOSU (Sevkiyat/Teslimat Bilgileri)")
print("=" * 60)
if fulfillment is not None:
    display(fulfillment.head())
    print("\nSütun bilgileri:")
    print(fulfillment.dtypes)
else:
    print("Fulfillment tablosu bulunamadı.")

In [ ]:
# OrderHeader tablosu
print("ORDER HEADER TABLOSU (Sipariş Başlık Bilgileri)")
print("=" * 60)
if order_header is not None:
    display(order_header.head())
else:
    print("OrderHeader tablosu bulunamadı.")

In [ ]:
# OrderLine tablosu
print("ORDER LINE TABLOSU (Sipariş Satır Detayları)")
print("=" * 60)
if order_line is not None:
    display(order_line.head())
else:
    print("OrderLine tablosu bulunamadı.")

In [ ]:
# Product tablosu
print("PRODUCT TABLOSU (Ürün Bilgileri)")
print("=" * 60)
if product is not None:
    display(product.head())
else:
    print("Product tablosu bulunamadı.")

In [ ]:
# Diğer tabloları da gösterelim
for isim in ['Promotion', 'Returns', 'Calendar']:
    tablo = tablolar.get(isim)
    if tablo is not None:
        print(f"\n{isim.upper()} TABLOSU")
        print("=" * 60)
        display(tablo.head(3))

---

## 3. Veri Ön İşleme

Bu bölümde verilerimizi analiz ve modelleme için hazır hale getiriyoruz.

### 3.1 Veri Tiplerinin Kontrolü

In [ ]:
# Fulfillment tablosunun detaylı bilgisi
print("Fulfillment Tablosu - Veri Tipleri ve Eksik Değerler")
print("=" * 60)
if fulfillment is not None:
    print(fulfillment.info())
    print("\n")
    print("Eksik değer sayıları:")
    print(fulfillment.isnull().sum())

### 3.2 Tarih Alanlarının Datetime Formatına Dönüştürülmesi

In [ ]:
# Fulfillment tablosundaki tarih sütunlarını tespit edelim
if fulfillment is not None:
    # Olası tarih sütunları
    tarih_sutunlari = [col for col in fulfillment.columns if 'Date' in col or 'date' in col]
    print("Tespit edilen tarih sütunları:", tarih_sutunlari)
    
    # Tarih sütunlarını datetime formatına çevirelim
    for sutun in tarih_sutunlari:
        if sutun in fulfillment.columns:
            fulfillment[sutun] = pd.to_datetime(fulfillment[sutun], errors='coerce')
            print(f"'{sutun}' sütunu datetime formatına dönüştürüldü.")
    
    print("\nGüncellenmiş veri tipleri:")
    print(fulfillment.dtypes)

### 3.3 Tabloların Birleştirilmesi (Merge)

Analiz için gerekli tabloları ortak anahtarlar üzerinden birleştiriyoruz.

In [ ]:
# Önce mevcut sütun isimlerini kontrol edelim
print("Fulfillment sütunları:", fulfillment.columns.tolist() if fulfillment is not None else "Yok")
print("\nOrderHeader sütunları:", order_header.columns.tolist() if order_header is not None else "Yok")
print("\nOrderLine sütunları:", order_line.columns.tolist() if order_line is not None else "Yok")

In [ ]:
# Ana veri setimizi oluşturalım
# Fulfillment tablosu ana tablomuz olacak çünkü teslimat durumu burada

df = fulfillment.copy()

# OrderHeader ile birleştirme (eğer ortak anahtar varsa)
if order_header is not None:
    # Ortak sütunu bulalım
    ortak_sutunlar = set(df.columns) & set(order_header.columns)
    print(f"Fulfillment ve OrderHeader ortak sütunları: {ortak_sutunlar}")
    
    # OrderID veya benzeri bir anahtar varsa birleştirelim
    for anahtar in ['OrderID', 'Order ID', 'OrderId', 'order_id']:
        if anahtar in df.columns and anahtar in order_header.columns:
            df = df.merge(order_header, on=anahtar, how='left', suffixes=('', '_order'))
            print(f"OrderHeader tablosu '{anahtar}' üzerinden birleştirildi.")
            break

print(f"\nBirleştirilmiş veri seti boyutu: {df.shape}")
print(f"Sütunlar: {df.columns.tolist()}")

In [ ]:
# Veri setinin genel görünümü
print("Birleştirilmiş Veri Seti - İlk 5 Satır")
print("=" * 60)
display(df.head())

### 3.4 Eksik ve Tutarsız Verilerin İncelenmesi

In [ ]:
# Eksik değerlerin analizi
print("Eksik Değer Analizi")
print("=" * 60)

eksik_degerler = df.isnull().sum()
eksik_yuzde = (df.isnull().sum() / len(df)) * 100

eksik_tablo = pd.DataFrame({
    'Eksik Değer Sayısı': eksik_degerler,
    'Yüzde (%)': eksik_yuzde.round(2)
})

# Sadece eksik değeri olanları gösterelim
eksik_tablo = eksik_tablo[eksik_tablo['Eksik Değer Sayısı'] > 0]

if len(eksik_tablo) > 0:
    print(eksik_tablo.sort_values('Eksik Değer Sayısı', ascending=False))
else:
    print("Veri setinde eksik değer bulunmamaktadır.")

In [ ]:
# Sayısal değişkenlerin istatistikleri
print("Sayısal Değişkenlerin İstatistikleri")
print("=" * 60)
display(df.describe())

---

## 4. Özellik Mühendisliği (Feature Engineering)

Makine öğrenmesi modelimiz için yeni ve anlamlı özellikler türetiyoruz.

### 4.1 Taşıma Süresi Hesaplama

In [ ]:
# Tarih sütunlarını kontrol edelim
print("Mevcut tarih sütunları:")
tarih_sutunlari = [col for col in df.columns if 'Date' in col or 'date' in col]
print(tarih_sutunlari)

In [ ]:
# Taşıma süresi: Teslimat Tarihi - Sevkiyat Tarihi
# Sütun isimlerini veri setine göre ayarlayalım

# Olası sütun isimlerini kontrol edelim
ship_date_col = None
delivery_date_col = None
promised_date_col = None

for col in df.columns:
    col_lower = col.lower()
    if 'ship' in col_lower and 'date' in col_lower:
        ship_date_col = col
    if 'delivery' in col_lower and 'date' in col_lower:
        delivery_date_col = col
    if 'promise' in col_lower and 'date' in col_lower:
        promised_date_col = col

print(f"Sevkiyat Tarihi Sütunu: {ship_date_col}")
print(f"Teslimat Tarihi Sütunu: {delivery_date_col}")
print(f"Taahhüt Tarihi Sütunu: {promised_date_col}")

In [ ]:
# Yeni özellikler oluşturalım

# 1. Taşıma Süresi (gün)
if ship_date_col and delivery_date_col:
    df['TasimaSuresi'] = (df[delivery_date_col] - df[ship_date_col]).dt.days
    print("'TasimaSuresi' özelliği oluşturuldu.")

# 2. Taahhüt Edilen Süre (gün)
if ship_date_col and promised_date_col:
    df['TaahhutSuresi'] = (df[promised_date_col] - df[ship_date_col]).dt.days
    print("'TaahhutSuresi' özelliği oluşturuldu.")

# 3. Gecikme Gün Sayısı
if delivery_date_col and promised_date_col:
    df['GecikmeGunu'] = (df[delivery_date_col] - df[promised_date_col]).dt.days
    # Negatif değerler erken teslimatı gösterir, bunları 0 yapalım
    df['GecikmeGunu'] = df['GecikmeGunu'].apply(lambda x: max(0, x) if pd.notna(x) else 0)
    print("'GecikmeGunu' özelliği oluşturuldu.")

print("\nYeni oluşturulan özelliklerin istatistikleri:")
yeni_ozellikler = ['TasimaSuresi', 'TaahhutSuresi', 'GecikmeGunu']
mevcut_ozellikler = [col for col in yeni_ozellikler if col in df.columns]
if mevcut_ozellikler:
    display(df[mevcut_ozellikler].describe())

### 4.2 Tarihlerden Zaman Bilgisi Çıkarma

In [ ]:
# Sevkiyat tarihinden ay, hafta ve gün bilgisi çıkaralım
if ship_date_col:
    df['SevkiyatAyi'] = df[ship_date_col].dt.month
    df['SevkiyatHaftasi'] = df[ship_date_col].dt.isocalendar().week
    df['SevkiyatGunu'] = df[ship_date_col].dt.dayofweek  # 0=Pazartesi, 6=Pazar
    df['HaftaSonu'] = df['SevkiyatGunu'].apply(lambda x: 1 if x >= 5 else 0)
    
    print("Tarih özellikleri oluşturuldu:")
    print("- SevkiyatAyi")
    print("- SevkiyatHaftasi")
    print("- SevkiyatGunu")
    print("- HaftaSonu")

display(df[['SevkiyatAyi', 'SevkiyatHaftasi', 'SevkiyatGunu', 'HaftaSonu']].head(10))

In [ ]:
# Güncellenmiş veri setinin yapısı
print(f"Güncellenmiş veri seti boyutu: {df.shape}")
print(f"\nTüm sütunlar:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2}. {col}")

---

## 5. Keşifsel Veri Analizi (EDA)

Verilerimizi görselleştirerek anlamlı örüntüler keşfediyoruz.

### 5.1 Teslimat Durumu Dağılımı

In [ ]:
# Teslimat durumu sütununu bulalım
delivery_status_col = None
for col in df.columns:
    if 'delivery' in col.lower() and 'status' in col.lower():
        delivery_status_col = col
        break
    elif 'status' in col.lower():
        delivery_status_col = col

if delivery_status_col:
    print(f"Teslimat durumu sütunu: {delivery_status_col}")
    print(f"\nTeslimat durumu dağılımı:")
    print(df[delivery_status_col].value_counts())
else:
    print("Teslimat durumu sütunu bulunamadı.")
    print("Mevcut sütunlar:", df.columns.tolist())

In [ ]:
# Teslimat durumu pasta grafiği
if delivery_status_col:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Pasta grafiği
    durum_dagilimi = df[delivery_status_col].value_counts()
    colors = ['#2ecc71', '#e74c3c'] if len(durum_dagilimi) == 2 else plt.cm.Set3.colors[:len(durum_dagilimi)]
    
    axes[0].pie(durum_dagilimi.values, labels=durum_dagilimi.index, autopct='%1.1f%%', 
                colors=colors, explode=[0.02]*len(durum_dagilimi), shadow=True)
    axes[0].set_title('Teslimat Durumu Dağılımı', fontsize=14, fontweight='bold')
    
    # Bar grafiği
    bars = axes[1].bar(durum_dagilimi.index, durum_dagilimi.values, color=colors, edgecolor='black')
    axes[1].set_xlabel('Teslimat Durumu', fontsize=12)
    axes[1].set_ylabel('Sipariş Sayısı', fontsize=12)
    axes[1].set_title('Teslimat Durumuna Göre Sipariş Sayısı', fontsize=14, fontweight='bold')
    
    # Bar üzerine değerleri yazalım
    for bar, val in zip(bars, durum_dagilimi.values):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
                     f'{val:,}', ha='center', va='bottom', fontsize=11)
    
    plt.tight_layout()
    plt.show()

### 5.2 Kargo Firmalarına Göre Gecikme Oranları

In [ ]:
# Kargo firması sütununu bulalım
carrier_col = None
for col in df.columns:
    col_lower = col.lower()
    if 'carrier' in col_lower or 'shipper' in col_lower or 'kargo' in col_lower:
        carrier_col = col
        break

if carrier_col:
    print(f"Kargo firması sütunu: {carrier_col}")
    print(f"\nKargo firmaları:")
    print(df[carrier_col].value_counts())
else:
    print("Kargo firması sütunu bulunamadı.")

In [ ]:
# Kargo firmalarına göre gecikme analizi
if carrier_col and delivery_status_col:
    # Çapraz tablo oluşturalım
    capraz_tablo = pd.crosstab(df[carrier_col], df[delivery_status_col], normalize='index') * 100
    
    print("Kargo Firmalarına Göre Teslimat Durumu (%)")
    print("=" * 50)
    display(capraz_tablo.round(2))
    
    # Görselleştirme
    fig, ax = plt.subplots(figsize=(12, 6))
    capraz_tablo.plot(kind='bar', ax=ax, colormap='RdYlGn', edgecolor='black')
    
    ax.set_xlabel('Kargo Firması', fontsize=12)
    ax.set_ylabel('Yüzde (%)', fontsize=12)
    ax.set_title('Kargo Firmalarına Göre Teslimat Performansı', fontsize=14, fontweight='bold')
    ax.legend(title='Teslimat Durumu', loc='upper right')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    
    # Yüzde çizgisi
    ax.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

### 5.3 Hizmet Seviyelerine Göre Teslimat Performansı

In [ ]:
# Hizmet seviyesi sütununu bulalım
service_col = None
for col in df.columns:
    col_lower = col.lower()
    if 'service' in col_lower or 'level' in col_lower or 'type' in col_lower:
        service_col = col
        break

if service_col:
    print(f"Hizmet seviyesi sütunu: {service_col}")
    print(f"\nHizmet seviyeleri:")
    print(df[service_col].value_counts())

In [ ]:
# Hizmet seviyesine göre teslimat performansı
if service_col and delivery_status_col:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Sol: Hizmet seviyesi dağılımı
    service_counts = df[service_col].value_counts()
    axes[0].bar(service_counts.index, service_counts.values, color='steelblue', edgecolor='black')
    axes[0].set_xlabel('Hizmet Seviyesi', fontsize=12)
    axes[0].set_ylabel('Sipariş Sayısı', fontsize=12)
    axes[0].set_title('Hizmet Seviyesi Dağılımı', fontsize=14, fontweight='bold')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Sağ: Hizmet seviyesine göre gecikme oranı
    gecikme_orani = df.groupby(service_col)[delivery_status_col].apply(
        lambda x: (x == 'Late').sum() / len(x) * 100 if 'Late' in x.values else 0
    ).sort_values(ascending=False)
    
    colors = ['#e74c3c' if val > 50 else '#f39c12' if val > 25 else '#2ecc71' for val in gecikme_orani.values]
    axes[1].barh(gecikme_orani.index, gecikme_orani.values, color=colors, edgecolor='black')
    axes[1].set_xlabel('Gecikme Oranı (%)', fontsize=12)
    axes[1].set_ylabel('Hizmet Seviyesi', fontsize=12)
    axes[1].set_title('Hizmet Seviyesine Göre Gecikme Oranı', fontsize=14, fontweight='bold')
    axes[1].axvline(x=50, color='red', linestyle='--', alpha=0.5, label='%50 eşik')
    
    plt.tight_layout()
    plt.show()

### 5.4 Zaman ve Sezon Etkilerinin İncelenmesi

In [ ]:
# Aylara göre sipariş ve gecikme analizi
if 'SevkiyatAyi' in df.columns and delivery_status_col:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Aylara göre sipariş sayısı
    ay_isimleri = ['Oca', 'Şub', 'Mar', 'Nis', 'May', 'Haz', 'Tem', 'Ağu', 'Eyl', 'Eki', 'Kas', 'Ara']
    aylik_siparis = df.groupby('SevkiyatAyi').size()
    
    axes[0, 0].bar(aylik_siparis.index, aylik_siparis.values, color='steelblue', edgecolor='black')
    axes[0, 0].set_xlabel('Ay', fontsize=11)
    axes[0, 0].set_ylabel('Sipariş Sayısı', fontsize=11)
    axes[0, 0].set_title('Aylara Göre Sipariş Sayısı', fontsize=13, fontweight='bold')
    axes[0, 0].set_xticks(range(1, 13))
    axes[0, 0].set_xticklabels(ay_isimleri, rotation=45)
    
    # 2. Aylara göre gecikme oranı
    aylik_gecikme = df.groupby('SevkiyatAyi')[delivery_status_col].apply(
        lambda x: (x == 'Late').sum() / len(x) * 100 if 'Late' in x.values else 0
    )
    
    axes[0, 1].plot(aylik_gecikme.index, aylik_gecikme.values, marker='o', linewidth=2, 
                    color='#e74c3c', markersize=8)
    axes[0, 1].fill_between(aylik_gecikme.index, aylik_gecikme.values, alpha=0.3, color='#e74c3c')
    axes[0, 1].set_xlabel('Ay', fontsize=11)
    axes[0, 1].set_ylabel('Gecikme Oranı (%)', fontsize=11)
    axes[0, 1].set_title('Aylara Göre Gecikme Oranı', fontsize=13, fontweight='bold')
    axes[0, 1].set_xticks(range(1, 13))
    axes[0, 1].set_xticklabels(ay_isimleri, rotation=45)
    axes[0, 1].axhline(y=50, color='gray', linestyle='--', alpha=0.5)
    
    # 3. Haftanın günlerine göre sipariş
    gun_isimleri = ['Pzt', 'Sal', 'Çar', 'Per', 'Cum', 'Cmt', 'Paz']
    gunluk_siparis = df.groupby('SevkiyatGunu').size()
    
    colors = ['#3498db']*5 + ['#e74c3c']*2  # Hafta içi mavi, hafta sonu kırmızı
    axes[1, 0].bar(gunluk_siparis.index, gunluk_siparis.values, color=colors, edgecolor='black')
    axes[1, 0].set_xlabel('Gün', fontsize=11)
    axes[1, 0].set_ylabel('Sipariş Sayısı', fontsize=11)
    axes[1, 0].set_title('Haftanın Günlerine Göre Sipariş Sayısı', fontsize=13, fontweight='bold')
    axes[1, 0].set_xticks(range(7))
    axes[1, 0].set_xticklabels(gun_isimleri)
    
    # 4. Hafta sonu vs hafta içi gecikme karşılaştırması
    haftasonu_analiz = df.groupby('HaftaSonu')[delivery_status_col].apply(
        lambda x: (x == 'Late').sum() / len(x) * 100 if 'Late' in x.values else 0
    )
    
    labels = ['Hafta İçi', 'Hafta Sonu']
    colors = ['#3498db', '#e74c3c']
    axes[1, 1].bar(labels, haftasonu_analiz.values, color=colors, edgecolor='black')
    axes[1, 1].set_ylabel('Gecikme Oranı (%)', fontsize=11)
    axes[1, 1].set_title('Hafta İçi vs Hafta Sonu Gecikme Oranı', fontsize=13, fontweight='bold')
    
    for i, val in enumerate(haftasonu_analiz.values):
        axes[1, 1].text(i, val + 1, f'{val:.1f}%', ha='center', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

### 5.5 Taşıma Süresi Analizi

In [ ]:
# Taşıma süresi dağılımı
if 'TasimaSuresi' in df.columns and delivery_status_col:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    axes[0].hist(df['TasimaSuresi'].dropna(), bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    axes[0].set_xlabel('Taşıma Süresi (gün)', fontsize=12)
    axes[0].set_ylabel('Frekans', fontsize=12)
    axes[0].set_title('Taşıma Süresi Dağılımı', fontsize=14, fontweight='bold')
    axes[0].axvline(df['TasimaSuresi'].mean(), color='red', linestyle='--', label=f'Ortalama: {df["TasimaSuresi"].mean():.1f} gün')
    axes[0].legend()
    
    # Teslimat durumuna göre boxplot
    df.boxplot(column='TasimaSuresi', by=delivery_status_col, ax=axes[1])
    axes[1].set_xlabel('Teslimat Durumu', fontsize=12)
    axes[1].set_ylabel('Taşıma Süresi (gün)', fontsize=12)
    axes[1].set_title('Teslimat Durumuna Göre Taşıma Süresi', fontsize=14, fontweight='bold')
    plt.suptitle('')  # Pandas'ın otomatik başlığını kaldır
    
    plt.tight_layout()
    plt.show()

### 5.6 Korelasyon Analizi

In [ ]:
# Sayısal değişkenler arasındaki korelasyon
sayisal_sutunlar = df.select_dtypes(include=[np.number]).columns.tolist()

# ID sütunlarını çıkaralım
sayisal_sutunlar = [col for col in sayisal_sutunlar if 'ID' not in col and 'Id' not in col]

if len(sayisal_sutunlar) > 1:
    korelasyon = df[sayisal_sutunlar].corr()
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(korelasyon, annot=True, cmap='RdYlBu_r', center=0, 
                fmt='.2f', square=True, linewidths=0.5)
    plt.title('Sayısal Değişkenler Arası Korelasyon Matrisi', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

---

## 6. Makine Öğrenmesi

### 6.1 Problem Tanımı

**Problem Türü:** İkili Sınıflandırma (Binary Classification)

**Hedef Değişken:** DeliveryStatus (Late / OnTime)

**Amaç:** Bir siparişin teslimatının geç olup olmayacağını tahmin etmek.

### 6.2 Veri Hazırlığı

In [ ]:
# Hedef değişkeni belirleyelim
if delivery_status_col:
    print(f"Hedef değişken: {delivery_status_col}")
    print(f"\nHedef değişken dağılımı:")
    print(df[delivery_status_col].value_counts())
    
    # Hedef değişkeni sayısala çevirelim (Late=1, OnTime=0)
    df['HedefDegisken'] = (df[delivery_status_col] == 'Late').astype(int)
    print(f"\nSayısallaştırılmış hedef değişken:")
    print(df['HedefDegisken'].value_counts())

In [ ]:
# Model için kullanılacak özellikleri seçelim
# ID sütunlarını ve tarih sütunlarını çıkaralım

# Potansiyel özellik sütunları
ozellik_adaylari = []

for col in df.columns:
    # ID ve tarih sütunlarını atla
    if 'ID' in col or 'Id' in col or 'id' in col:
        continue
    if 'Date' in col or 'date' in col:
        continue
    if col == delivery_status_col or col == 'HedefDegisken':
        continue
    
    ozellik_adaylari.append(col)

print("Potansiyel özellik sütunları:")
for i, col in enumerate(ozellik_adaylari, 1):
    print(f"{i:2}. {col} - Tip: {df[col].dtype}")

In [ ]:
# Kategorik değişkenleri sayısallaştıralım
from sklearn.preprocessing import LabelEncoder

# Model için kullanılacak DataFrame'i oluşturalım
df_model = df.copy()

# Kategorik sütunları tespit edelim
kategorik_sutunlar = df_model[ozellik_adaylari].select_dtypes(include=['object']).columns.tolist()
print(f"Kategorik sütunlar: {kategorik_sutunlar}")

# Label Encoding uygulayalım
label_encoders = {}
for col in kategorik_sutunlar:
    le = LabelEncoder()
    df_model[col + '_encoded'] = le.fit_transform(df_model[col].astype(str))
    label_encoders[col] = le
    print(f"'{col}' sütunu encode edildi. Sınıflar: {le.classes_}")

In [ ]:
# Final özellik setini oluşturalım
# Sayısal özellikler + encode edilmiş kategorik özellikler

sayisal_ozellikler = []
for col in ozellik_adaylari:
    if df_model[col].dtype in ['int64', 'float64', 'int32', 'float32']:
        sayisal_ozellikler.append(col)

encoded_ozellikler = [col + '_encoded' for col in kategorik_sutunlar]

final_ozellikler = sayisal_ozellikler + encoded_ozellikler

print("Model için kullanılacak özellikler:")
print("-" * 40)
for i, col in enumerate(final_ozellikler, 1):
    print(f"{i:2}. {col}")

In [ ]:
# X (özellikler) ve y (hedef) değişkenlerini ayıralım
X = df_model[final_ozellikler].copy()
y = df_model['HedefDegisken'].copy()

# Eksik değerleri dolduralım
X = X.fillna(X.median())

print(f"X (Özellikler) boyutu: {X.shape}")
print(f"y (Hedef) boyutu: {y.shape}")
print(f"\nHedef değişken dağılımı:")
print(y.value_counts())

### 6.3 Eğitim ve Test Setlerinin Oluşturulması

In [ ]:
# Veriyi eğitim (%80) ve test (%20) olarak ayıralım
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.20,  # %20 test seti
    random_state=42,  # Tekrarlanabilirlik için
    stratify=y  # Sınıf dengesini korumak için
)

print("Veri Seti Bölümleme")
print("=" * 40)
print(f"Eğitim seti boyutu: {X_train.shape[0]} örnek")
print(f"Test seti boyutu: {X_test.shape[0]} örnek")
print(f"\nEğitim seti hedef dağılımı:")
print(y_train.value_counts())
print(f"\nTest seti hedef dağılımı:")
print(y_test.value_counts())

### 6.4 Model Kurulumu - Lojistik Regresyon

In [ ]:
# Lojistik Regresyon modeli oluşturalım
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Modeli eğitelim
log_reg.fit(X_train, y_train)

# Tahmin yapalım
y_pred_log = log_reg.predict(X_test)

print("Lojistik Regresyon modeli eğitildi!")

### 6.5 Model Kurulumu - Karar Ağacı

In [ ]:
# Karar Ağacı modeli oluşturalım
dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)

# Modeli eğitelim
dt_model.fit(X_train, y_train)

# Tahmin yapalım
y_pred_dt = dt_model.predict(X_test)

print("Karar Ağacı modeli eğitildi!")

---

## 7. Model Değerlendirme

### 7.1 Accuracy (Doğruluk) Metriği

In [ ]:
# Her iki modelin doğruluk skorları
acc_log = accuracy_score(y_test, y_pred_log)
acc_dt = accuracy_score(y_test, y_pred_dt)

print("Model Doğruluk Skorları")
print("=" * 40)
print(f"Lojistik Regresyon: {acc_log:.4f} ({acc_log*100:.2f}%)")
print(f"Karar Ağacı: {acc_dt:.4f} ({acc_dt*100:.2f}%)")

In [ ]:
# Doğruluk skorlarını görselleştirelim
modeller = ['Lojistik Regresyon', 'Karar Ağacı']
skorlar = [acc_log, acc_dt]

fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.bar(modeller, skorlar, color=['#3498db', '#2ecc71'], edgecolor='black')

ax.set_ylabel('Doğruluk Skoru', fontsize=12)
ax.set_title('Model Karşılaştırması - Doğruluk Skorları', fontsize=14, fontweight='bold')
ax.set_ylim(0, 1)

# Bar üzerine değerleri yazalım
for bar, skor in zip(bars, skorlar):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{skor*100:.2f}%', ha='center', va='bottom', fontsize=14, fontweight='bold')

# Referans çizgisi
ax.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Rastgele tahmin (%50)')
ax.legend()

plt.tight_layout()
plt.show()

### 7.2 Confusion Matrix (Karışıklık Matrisi)

In [ ]:
# Her iki model için confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Lojistik Regresyon
cm_log = confusion_matrix(y_test, y_pred_log)
sns.heatmap(cm_log, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['OnTime (0)', 'Late (1)'],
            yticklabels=['OnTime (0)', 'Late (1)'])
axes[0].set_xlabel('Tahmin Edilen', fontsize=12)
axes[0].set_ylabel('Gerçek', fontsize=12)
axes[0].set_title(f'Lojistik Regresyon\nDoğruluk: {acc_log*100:.2f}%', fontsize=13, fontweight='bold')

# Karar Ağacı
cm_dt = confusion_matrix(y_test, y_pred_dt)
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['OnTime (0)', 'Late (1)'],
            yticklabels=['OnTime (0)', 'Late (1)'])
axes[1].set_xlabel('Tahmin Edilen', fontsize=12)
axes[1].set_ylabel('Gerçek', fontsize=12)
axes[1].set_title(f'Karar Ağacı\nDoğruluk: {acc_dt*100:.2f}%', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

### 7.3 Detaylı Sınıflandırma Raporu

In [ ]:
# Lojistik Regresyon için sınıflandırma raporu
print("LOJİSTİK REGRESYON - Sınıflandırma Raporu")
print("=" * 55)
print(classification_report(y_test, y_pred_log, target_names=['OnTime', 'Late']))

In [ ]:
# Karar Ağacı için sınıflandırma raporu
print("KARAR AĞACI - Sınıflandırma Raporu")
print("=" * 55)
print(classification_report(y_test, y_pred_dt, target_names=['OnTime', 'Late']))

### 7.4 Özellik Önemliliği (Feature Importance)

In [ ]:
# Karar Ağacı modelinin özellik önemliliği
ozellik_onemliligi = pd.DataFrame({
    'Ozellik': final_ozellikler,
    'Onem': dt_model.feature_importances_
}).sort_values('Onem', ascending=True)

plt.figure(figsize=(10, 8))
plt.barh(ozellik_onemliligi['Ozellik'], ozellik_onemliligi['Onem'], color='steelblue', edgecolor='black')
plt.xlabel('Önem Derecesi', fontsize=12)
plt.ylabel('Özellik', fontsize=12)
plt.title('Karar Ağacı - Özellik Önemliliği', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nEn önemli 5 özellik:")
print(ozellik_onemliligi.tail().to_string(index=False))

---

## 8. Sonuç ve Yorum

### 8.1 Model Performansının Değerlendirilmesi

In [ ]:
# Sonuç özeti
print("="*70)
print("                    SONUÇ ÖZETİ                    ")
print("="*70)
print(f"""
1. VERİ SETİ
   - Toplam kayıt sayısı: {len(df):,}
   - Kullanılan özellik sayısı: {len(final_ozellikler)}
   - Eğitim seti: {len(X_train):,} örnek
   - Test seti: {len(X_test):,} örnek

2. MODEL PERFORMANSLARI
   - Lojistik Regresyon Doğruluğu: {acc_log*100:.2f}%
   - Karar Ağacı Doğruluğu: {acc_dt*100:.2f}%
   
3. EN İYİ MODEL: {'Lojistik Regresyon' if acc_log > acc_dt else 'Karar Ağacı'}
   - Doğruluk: {max(acc_log, acc_dt)*100:.2f}%
""")
print("="*70)

### 8.2 Teslimat Gecikmesini Etkileyen Faktörler

In [ ]:
# En önemli faktörlerin yorumu
print("Teslimat Gecikmesini Etkileyen Temel Faktörler")
print("="*55)

en_onemli = ozellik_onemliligi.tail(5).iloc[::-1]
for i, (_, row) in enumerate(en_onemli.iterrows(), 1):
    print(f"{i}. {row['Ozellik']}: {row['Onem']*100:.2f}% önem")

### 8.3 Veri Setinin Güçlü ve Sınırlı Yönleri

**Güçlü Yönler:**
- Çoklu tablo yapısı sayesinde kapsamlı bir e-ticaret senaryosu sunması
- Black Friday, Cyber Monday ve Noel gibi yoğun dönemleri içermesi
- Teslimat, iade ve kampanya bilgilerinin bir arada bulunması
- Gerçekçi lojistik verileri içermesi

**Sınırlı Yönler:**
- Coğrafi bilgilerin (mesafe, bölge) sınırlı olması
- Hava durumu gibi dış faktörlerin bulunmaması
- Müşteri davranış verilerinin eksikliği

---

## Kaynaklar

- Veri Seti: Kaggle - Christmas Retail Sales – Shipping & Delivery Dataset
- Python Kütüphaneleri: pandas, numpy, matplotlib, seaborn, scikit-learn

---

*Bu notebook, Veri Bilimine Giriş ve Makine Öğrenmesi dersi kapsamında hazırlanmıştır.*